<a href="https://colab.research.google.com/github/cjudithrb/NLP-FineTuning-PEFT-Classification/blob/main/Fine_tuning_Clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1: Pre-requisitos**

### **1.1. Instalación de Dependencias**

In [ ]:
import subprocess
# Instala todas las dependencias necesarias
!pip install transformers
!pip install accelerate
#!pip install mlflow==2.16.0 pyngrok==7.2.0

In [ ]:
!pip install wandb

  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached sentry_sdk-2.51.0-py2.py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install evaluate==0.4.0 rouge-score==0.1.2
!pip install peft==0.13.2 datasets==3.0.1 wandb==0.13.1

  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


### **1.2: Importación de Librerías**

In [ ]:
import torch
import subprocess
import getpass # Para autenticarse en HF
import pandas as pd
import numpy as np
import datetime
import pytz # Zona horaria
import wandb
import re
import os
from huggingface_hub import HfApi, snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification #, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback, GenerationConfig
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel, PeftConfig

## **2: Configuración**

### **2.1. Definicion de variables**

In [ ]:
def clean_model_name(name):
    """Limpia el nombre del modelo, eliminando caracteres no permitidos para Hugging Face Hub."""
    # Reemplazar espacios y puntos con guiones bajos
    name = name.replace(" ", "_").replace(".", "_")
    # Eliminar cualquier otro carácter especial no permitido
    name = re.sub(r'[^a-zA-Z0-9_-]', '', name)
    return name

In [ ]:
import os
BASE_MODEL_NAME = "Qwen/Qwen2.5-0.5B"  # Qwen/Qwen3-0.6B dccuchile/bert-base-spanish-wwm-cased / UMUTeam/emotions-DistilBETO  #Qwen/Qwen2.5-1.5B / meta-llama/Llama-3.2-1B / microsoft/phi-1
MODEL_NAME = BASE_MODEL_NAME.split('/')[-1]
CLEAN_MODEL_NAME = clean_model_name(MODEL_NAME)
print(MODEL_NAME)
print(CLEAN_MODEL_NAME)

# Habilitar descargas más rápidas de Hugging Face
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

#FILE_PATH = '/content/drive/MyDrive/ColabNotebooks/Data/dataset10k.tsv'
#FILE_NAME = os.path.splitext(os.path.basename(FILE_PATH))[0]
#print(FILE_NAME)

Qwen2.5-0.5B
Qwen2_5-0_5B


In [ ]:
CLEAN_MODEL_NAME="DistilBETO"
CLEAN_MODEL_NAME

'DistilBETO'

### **2.2. Login Hugging Face**

In [ ]:
def set_huggingface_hub_token():
    """Configura el token para Hugging Face Hub solicitándolo al usuario y verificando su existencia."""
    if "HUGGING_FACE_HUB_TOKEN" in os.environ:
        print("Token ya configurado.")
    else:
        token = getpass.getpass("Token: ")
        if token:  # Verificar que el token no esté vacío
            os.environ["HUGGING_FACE_HUB_TOKEN"] = token
            print("Token configurado exitosamente.")
        else:
            raise ValueError("No se ingresó un token válido.")

# Uso de la función
try:
    set_huggingface_hub_token()
except ValueError as e:
    print(e)

Token: ··········
Token configurado exitosamente.


In [ ]:
from google.colab import output
output.secret("HF_TOKEN")

### **2.3. Configuración del Modelo**

In [ ]:
def get_device():
    """Configura y retorna el dispositivo basado en la disponibilidad de CUDA."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")
    return device

def set_device():
    """Configura y retorna el dispositivo para entrenamiento basado en la disponibilidad de CUDA."""
    if torch.cuda.is_available():
        print("CUDA está disponible. Usando GPU.")
        return torch.device("cuda")
    else:
        print("CUDA no está disponible. Usando CPU.")
        return torch.device("cpu")

In [ ]:
def load_model_and_tokenizer(base_model_name):
    """Carga un modelo y un tokenizador para clasificación de secuencias."""
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            base_model_name,
            num_labels=2,
            device_map="auto",
            attn_implementation='eager'
        )
        tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True, clean_up_tokenization_spaces=True)

        # Configurar pad_token como eos_token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Sincronizar la configuración del modelo
        model.config.pad_token_id = tokenizer.pad_token_id

        print(f"Modelo y tokenizador cargados. Pad token ID: {model.config.pad_token_id}")
        return model, tokenizer
    except Exception as e:
        print(f"Error al cargar el modelo o el tokenizador: {e}")
        return None, None

model_base, tokenizer = load_model_and_tokenizer(BASE_MODEL_NAME)

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo (SequenceClassification) y tokenizador cargados exitosamente.


## **3: Carga y Preparación del Dataset**

In [ ]:
from datasets import load_dataset

# 1. Cargar el dataset desde Hugging Face
dataset = load_dataset("PrevenIA/spanish-suicide-intent")

# 2. Revisar las divisiones disponibles
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-7b34565378f029(…):   0%|          | 0.00/18.0M [00:00<?, ?B/s]

data/val-00000-of-00001-d7338c59b5e5031f(…):   0%|          | 0.00/2.03M [00:00<?, ?B/s]

data/test-00000-of-00001-c830a979da438bf(…):   0%|          | 0.00/4.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/136136 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/15131 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/37820 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text', 'Label', 'dataset', '__index_level_0__'],
        num_rows: 136136
    })
    val: Dataset({
        features: ['Text', 'Label', 'dataset', '__index_level_0__'],
        num_rows: 15131
    })
    test: Dataset({
        features: ['Text', 'Label', 'dataset', '__index_level_0__'],
        num_rows: 37820
    })
})

In [ ]:
df_test=dataset['test'].to_pandas()
df_val=dataset['val'].to_pandas()

df_test.head()

,Text,Label,dataset,__index_level_0__
0,Me voy a matar hoy mi ex novio me odia y todav...,1,Hackaton Somos NLP,8025
1,Genera una oración usando las siguientes parte...,0,Hackaton Somos NLP,9273
2,Necesito un artículo de blog sobre cómo las re...,0,Hackaton Somos NLP,2836
3,Crear una pregunta de opción múltiple sobre nu...,0,Hackaton Somos NLP,690
4,Haz una sugerencia para resolver el siguiente ...,0,Hackaton Somos NLP,6234


### Probar inferencia

In [ ]:
index = 1
original_text = dataset['test'][index]['Text']
label_numeric = dataset['test'][index]['Label']
label_text = "Sí" if label_numeric == 1 else "No"

# Tokenización
inputs = tokenizer(original_text, return_tensors='pt', truncation=True, padding=True)
inputs = inputs.to(model_base.device)

# Inferencia para clasificación
with torch.no_grad():
    outputs = model_base(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()

generated_label = "Sí tiene intención suicida" if prediction == 1 else "No tiene intención suicida"

print('-' * 80)
print(f'Example Index: {index}')
print('-' * 80)
print(f'ORIGINAL INPUT TEXT:\n{original_text}')
print('-' * 80)
print(f'BASELINE HUMAN CLASSIFICATION:\n{label_text} (Numeric Label: {label_numeric})')
print('-' * 80)
print(f'MODEL PREDICTION:\n{generated_label}\n')

--------------------------------------------------------------------------------
Example Index: 1
--------------------------------------------------------------------------------
ORIGINAL INPUT TEXT:
Genera una oración usando las siguientes partes del discurso.
--------------------------------------------------------------------------------
BASELINE HUMAN CLASSIFICATION:
No (Numeric Label: 0)
--------------------------------------------------------------------------------
MODEL PREDICTION:
No tiene intención suicida



In [ ]:
df_test[df_test["Label"] == 1]

,Text,Label,dataset,__index_level_0__
0,Me voy a matar hoy mi ex novio me odia y todav...,1,Hackaton Somos NLP,8025
23,murmuriendo quiero morir para mí mismo diariam...,1,Hackaton Somos NLP,4876
24,Una noche más una mañana más un día más. Ya no...,1,Hackaton Somos NLP,6392
25,Ya no tengo nada por lo que vivir,1,Hackaton Somos NLP,431
28,"No sé qué hacer, me siento tan miserable que q...",1,Hackaton Somos NLP,7255
...,...,...,...,...
37811,Realmente quiero que esta parte de mi vida ter...,1,Suicidal Tweet Detection Dataset,26
37814,"Maldita sea, me odio a mí mismo @EvoPvPServer",1,Suicidal Tweet Detection Dataset,86
37817,Publicaré esto cada vez que aparezca y compart...,1,Suicidal Tweet Detection Dataset,1741
37818,¿Por qué regresé? Quiero morirme. Estoy cansad...,1,Suicidal Tweet Detection Dataset,1494


In [ ]:
def generate_test_case_example(index):
    """Genera y muestra un caso de prueba generado por el modelo basado en el índice proporcionado."""

    original_text = dataset['test'][index]['Text']
    label_numeric = dataset['test'][index]['Label']
    label_text = "Sí" if label_numeric == 1 else "No" # Convert numeric label to text ('Yes' for 1, 'No' for 0)

    # Construye el prompt para la clasificación de la intención suicida
    prompt = f"Clasifica el siguiente texto como 'Sí tien intención suicida' si indica intención suicida o 'No tiene intención suicida' en caso contrario: '{original_text}'\nClasificación:"

    # Tokeniza el prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Mueve los inputs al mismo dispositivo que el modelo (GPU)
    inputs = inputs.to(model_base.device)

    # El modelo genera una salida (clasificación) basada en el prompt
    # Se configuran los parámetros para una salida concisa y determinista
    output_ids = model_base.generate(
        inputs["input_ids"],  # IDs de los tokens de la entrada
        max_new_tokens=10,   # Limita el número de tokens generados para concisión (e.g., "Sí" or "No")
        do_sample=False,     # Para una salida más determinista en clasificación
        pad_token_id=tokenizer.eos_token_id # Importante para la generación cuando no hay pad_token explícito
    )[0]  # Selecciona la primera secuencia generada

    # Decodifica solo la parte generada, excluyendo el prompt original
    generated_text = tokenizer.decode(
        output_ids[len(inputs["input_ids"][0]):],  # Decodifica a partir del final de los input_ids
        skip_special_tokens=True  # Omite tokens especiales en la salida
    ).strip()

    # Imprime los resultados
    print('-' * 80)
    print(f'Example Index: {index}')
    print('-' * 80)
    print(f'ORIGINAL INPUT TEXT:\n{original_text}')
    print('-' * 80)
    #print(f'PROMPT SENT TO MODEL:\n{prompt}')
    #print('-' * 80)
    print(f'BASELINE HUMAN CLASSIFICATION:\n{label_text} (Numeric Label: {label_numeric})')
    print('-' * 80)
    print(f'MODEL GENERATED CLASSIFICATION:\n{generated_text}\n')


# Número de ejemplos que se desean mostrar
num_examples_to_show = 4

# Generar y mostrar el número de ejemplos especificados
for i in range(num_examples_to_show):
    generate_test_case_example(i)

--------------------------------------------------------------------------------
Example Index: 0
--------------------------------------------------------------------------------
ORIGINAL INPUT TEXT:
Me voy a matar hoy mi ex novio me odia y todavía estoy enamorado de él me odia por no poder lidiar con la ruptura de hora y hacerle preguntas para conseguir el cierre no puedo seguir tomando el odio y el dolor
--------------------------------------------------------------------------------
BASELINE HUMAN CLASSIFICATION:
Sí (Numeric Label: 1)
--------------------------------------------------------------------------------
MODEL GENERATED CLASSIFICATION:
No tiene intención suicida

La frase "

--------------------------------------------------------------------------------
Example Index: 1
--------------------------------------------------------------------------------
ORIGINAL INPUT TEXT:
Genera una oración usando las siguientes partes del discurso.
-----------------------------------------

## **4: Entrenamiento**

### **4.1. Tokenizacion**

In [ ]:
def tokenize_preprocess(examples, tokenizer, max_length=64):
    """Tokeniza los inputs y prepara las etiquetas para el modelo de clasificación de secuencias."""

    # Para clasificación de secuencias, simplemente tokenizamos el texto
    tokenized_data = tokenizer(
        examples['Text'],
        truncation=True,
        padding='max_length',
        max_length=max_length
    )

    # Las etiquetas para clasificación son los valores numéricos directamente
    tokenized_data['labels'] = examples['Label']

    return tokenized_data

def prepare_datasets(dataset: Dataset, tokenizer, test_size: float=0.1, validation_size: float=0.9):
    """Aplica la tokenización y divide el dataset en entrenamiento, prueba y validación."""
    # Para BertTokenizer, eos_token es None por defecto, lo cual puede causar problemas con padding en algunas operaciones.
    # Nos aseguramos de que pad_token esté configurado y, si eos_token es None, lo asignamos a pad_token.
    if tokenizer.pad_token is None:
        # Si '[PAD]' existe en el vocabulario, lo usamos. De lo contrario, usamos unk_token.
        if '[PAD]' in tokenizer.get_vocab():
            tokenizer.pad_token = '[PAD]'
        else:
            tokenizer.pad_token = tokenizer.unk_token
        # Aseguramos que pad_token_id también esté configurado si pad_token fue establecido
        if tokenizer.pad_token_id is None or tokenizer.pad_token_id < 0:
            tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

    # Si eos_token es None (común en BertTokenizer), lo asignamos a pad_token para consistencia en la generación.
    if tokenizer.eos_token is None:
        tokenizer.eos_token = tokenizer.pad_token
        tokenizer.eos_token_id = tokenizer.pad_token_id

    # Aplicar preprocesamiento al dataset
    tokenized_dataset = dataset.map(lambda x: tokenize_preprocess(x, tokenizer), batched=True)
    # Dividir el dataset en 80% train y 20% resto
    train_test_split = tokenized_dataset.train_test_split(test_size=test_size)
    test_validation_split = train_test_split['test'].train_test_split(test_size=validation_size)

    return {
        'train': train_test_split['train'],
        'test': test_validation_split['test'],
        'validation': test_validation_split['train']
    }

# Uso de las funciones
tokenized_datasets = prepare_datasets(dataset['test'], tokenizer)
print(tokenized_datasets)

Map:   0%|          | 0/37820 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['Text', 'Label', 'dataset', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 34038
}), 'test': Dataset({
    features: ['Text', 'Label', 'dataset', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3404
}), 'validation': Dataset({
    features: ['Text', 'Label', 'dataset', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 378
})}


In [ ]:
#FINAL
def tokenize_preprocess(examples, tokenizer, max_length=64):
    """Tokeniza los inputs y prepara las etiquetas."""
    tokenized_data = tokenizer(
        examples['Text'],
        truncation=True,
        padding='max_length',
        max_length=max_length
    )
    tokenized_data['labels'] = examples['Label']
    return tokenized_data

def prepare_datasets(dataset: Dataset, tokenizer, test_size: float=0.1, validation_size: float=0.9):
    """Aplica la tokenización y divide el dataset."""
    # Asegurar que el pad_token esté configurado antes de mapear
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenized_dataset = dataset.map(lambda x: tokenize_preprocess(x, tokenizer), batched=True)

    train_test_split = tokenized_dataset.train_test_split(test_size=test_size)
    test_validation_split = train_test_split['test'].train_test_split(test_size=validation_size)

    return {
        'train': train_test_split['train'],
        'test': test_validation_split['test'],
        'validation': test_validation_split['train']
    }

# Re-preparamos los datasets
tokenized_datasets = prepare_datasets(dataset['test'], tokenizer)
print(tokenized_datasets)

### **4.2. Configuración LoRA**

In [ ]:
def configure_lora(task_type, r, lora_alpha, lora_dropout):
    """Configura y retorna los ajustes de LoRA."""
    lora_config = LoraConfig(
        task_type=task_type,
        inference_mode=False,  # Modo de entrenamiento
        r=r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout
    )
    return lora_config

def apply_lora_to_model(model_base, lora_config):
    """Aplica LoRA al modelo y lo mueve al dispositivo especificado."""
    try:
        model = get_peft_model(model_base, lora_config)  # Asumiendo que esta función ya está definida
        #model = model.to(device)
        print("Modelo adaptado con LoRA y movido al dispositivo correctamente.")
        return model
    except Exception as e:
        print(f"Error al aplicar LoRA o mover el modelo al dispositivo: {e}")
        return None

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param} %"
    )
#print_trainable_parameters(peft_model_llama)

### **4.3. Configuración Finetuning PEFT**

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
from transformers import EarlyStoppingCallback

def init_wandb(training_args, project_name, model_name, file_name, current_datetime):
    """Inicializa la sesión de WandB."""
    wandb.init(
        project=project_name,
        name=f"Run_{model_name}_{current_datetime}",
        config=vars(training_args)
    )

class MetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            metrics_to_log = ['loss', 'eval_loss', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall', 'learning_rate', 'step', 'epoch']
            wandb.log({k: v for k, v in logs.items() if k in metrics_to_log}, step=state.global_step)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions),
        "precision": precision_score(labels, predictions),
        "recall": recall_score(labels, predictions),
    }

def train_model(model, tokenizer, training_args, train_dataset, eval_dataset, compute_metrics, MetricsLogger, model_name, output_dir, current_datetime):
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[MetricsLogger(), EarlyStoppingCallback(early_stopping_patience=3)]
    )
    try:
        trainer.train()
        trainer.model.save_pretrained(output_dir)
        trainer.save_model(output_dir)
    except Exception as e:
        print(f"Ha ocurrido un error durante el entrenamiento: {e}")

In [ ]:
import os

def configure_training():
    lima_tz = pytz.timezone('America/Lima')
    current_datetime = datetime.datetime.now(lima_tz).strftime("%Y%m%d_%H%M%S")
    output_dir = f"./results/run_{current_datetime}"
    print(output_dir)

    # Check for PYTORCH_CUDA_ALLOC_CONF environment variable
    if 'PYTORCH_CUDA_ALLOC_CONF' not in os.environ:
        print("Consider setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True if memory issues persist.")

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=2,
        learning_rate=0.001,
        report_to="none",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=16, # Aumentado a 4 para reducir la memoria por paso
        eval_strategy="steps",
        eval_steps=4,
        save_steps=4,
        logging_dir='./logs',
        logging_steps=4,
        fp16=True,
        bf16=False, # Explicitly disable bf16 as well
        gradient_checkpointing=False, # Desactivado para evitar el error 'element 0 of tensors does not require grad'
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
    )
    return training_args, output_dir, current_datetime

### **4.4. Ejecucion**

In [ ]:
PROJECT_NAME_WANDB="FinetuningClasificacion"
LORA_RANK = 4
LORA_ALPHA = 8
LORA_DROPOUT = 0.05
TASK_TYPE = TaskType.SEQ_CLS

lora_config = LoraConfig(
    task_type=TASK_TYPE,
    inference_mode=False,
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT
)

# Re-aplicamos LoRA al nuevo model_base de clasificación
peft_model = apply_lora_to_model(model_base, lora_config)
print_trainable_parameters(peft_model)

Modelo adaptado con LoRA y movido al dispositivo correctamente.
trainable params: 272128 || all params: 494306688 || trainable%: 0.05505246168144098 %


In [ ]:
if __name__ == "__main__":
    peft_training_args, output_dir, current_datetime = configure_training()
    init_wandb(peft_training_args, PROJECT_NAME_WANDB, MODEL_NAME, "data34k", current_datetime)
    # Pasamos el tokenizer a la función de entrenamiento
    train_model(peft_model, tokenizer, peft_training_args, tokenized_datasets['train'], tokenized_datasets['validation'], compute_metrics, MetricsLogger, MODEL_NAME, output_dir, current_datetime)

./results/run_20260201_225818
Consider setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True if memory issues persist.


/tmp/ipython-input-1457035300.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
2,44.755600,2.898280,0.465608,0.530233,0.465306,0.616216
4,41.413800,2.898280,0.465608,0.530233,0.465306,0.616216
6,46.170800,2.898280,0.465608,0.530233,0.465306,0.616216
8,44.802400,2.484771,0.494709,0.284644,0.463415,0.205405
10,28.998700,1.640257,0.507937,0.535000,0.497674,0.578378
12,24.416400,1.142751,0.584656,0.590078,0.570707,0.610811
14,16.512700,1.088018,0.589947,0.444444,0.659574,0.335135
16,15.598400,0.849109,0.693122,0.730233,0.640816,0.848649
18,15.013200,0.840688,0.701058,0.739030,0.645161,0.864865
20,13.442300,0.817335,0.701058,0.638978,0.781250,0.540541


In [ ]:
import torch
torch.cuda.empty_cache()  # Esto liberará la memoria no utilizada


In [ ]:
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
eval_accuracy,▁▃▃▅▅▅▆▆▆▆▇▇▇▇▇▇█▇▇▇██▇▇███▇▇███████████
eval_f1,▁▆▆▅▆▆▆▇▆▇▆▇▇▇▇▇▇▇██████████████████████
eval_loss,██▅▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_precision,▁▁▂▆▅▆▅▆▇▇▇▆▇▅▇▇▆█▇█▇██▆██▇▇▇███▇▇▇▇▇█▇▇
eval_recall,▄▁▃▆▆▄▆▆▆▆▆▇▆▇▆▇▆▆▇▇▇▇█▇▇██▇▇▇▇▇▆▇▇▇▇▇▇▇
learning_rate,███▇▇▇▇▇▇▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
loss,█▅▃▃▃▂▂▂▂▁▂▂▂▁▁▂▁▂▁▁▂▁▁▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch,2
eval_accuracy,0.91005
eval_f1,0.9071


## **5: Evaluación**

In [ ]:
import torch
import pandas as pd

# Verificamos si CUDA (GPU) está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Obtenemos los primeros 10 casos del conjunto de validación (o test)
num_samples = 20
texts = dataset['val'][0:num_samples]['Text']
human_baseline_labels = dataset['val'][0:num_samples]['Label']

base_model_predictions = []
peft_model_predictions = []
base_model_probabilities = []
peft_model_probabilities = []

# Movemos los modelos al dispositivo (GPU o CPU)
model_base.to(device)
peft_model.to(device)

for idx, text in enumerate(texts):
    # Tokenización
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # ========== MODELO BASE ==========
    with torch.no_grad():
        base_outputs = model_base(**inputs)
        base_logits = base_outputs.logits

        # Predicción (clase con mayor logit)
        base_prediction = torch.argmax(base_logits, dim=-1).item()

        # Probabilidades (softmax sobre logits)
        base_probs = torch.softmax(base_logits, dim=-1)
        base_prob_clase1 = base_probs[0, 1].item()  # Probabilidad de clase 1 (suicida)

    base_model_predictions.append(base_prediction)
    base_model_probabilities.append(base_prob_clase1)

    # ========== MODELO PEFT ==========
    with torch.no_grad():
        peft_outputs = peft_model(**inputs)
        peft_logits = peft_outputs.logits

        # Predicción
        peft_prediction = torch.argmax(peft_logits, dim=-1).item()

        # Probabilidades
        peft_probs = torch.softmax(peft_logits, dim=-1)
        peft_prob_clase1 = peft_probs[0, 1].item()

    peft_model_predictions.append(peft_prediction)
    peft_model_probabilities.append(peft_prob_clase1)

# Verificar que todas las listas tengan la misma longitud
if len(human_baseline_labels) == len(base_model_predictions) == len(peft_model_predictions):
    # Convertir etiquetas numéricas a texto
    human_labels_text = ["Sí" if label == 1 else "No" for label in human_baseline_labels]
    base_predictions_text = ["Sí" if pred == 1 else "No" for pred in base_model_predictions]
    peft_predictions_text = ["Sí" if pred == 1 else "No" for pred in peft_model_predictions]

    # Crear DataFrame con resultados
    df_report = pd.DataFrame({
        'TEXTO': texts,
        'ETIQUETA_REAL': human_labels_text,
        #'ETIQUETA_REAL_NUM': human_baseline_labels,
        'BASE_MODEL_PRED': base_predictions_text,
        #'BASE_MODEL_PROB': [f"{p:.3f}" for p in base_model_probabilities],
        f'PEFT_{MODEL_NAME.upper()}_PRED': peft_predictions_text,
        #f'PEFT_{MODEL_NAME.upper()}_PROB': [f"{p:.3f}" for p in peft_model_probabilities],
        'BASE_CORRECTA': [h == b for h, b in zip(human_baseline_labels, base_model_predictions)],
        'PEFT_CORRECTA': [h == p for h, p in zip(human_baseline_labels, peft_model_predictions)]
    })

    # Métricas de resumen
    base_accuracy = sum(df_report['BASE_CORRECTA']) / len(df_report)
    peft_accuracy = sum(df_report['PEFT_CORRECTA']) / len(df_report)

    print("=" * 80)
    print(f"COMPARACIÓN DE MODELOS - {num_samples} CASOS DE VALIDACIÓN")
    print("=" * 80)
    print(f"Accuracy Modelo Base: {base_accuracy:.2%}")
    print(f"Accuracy Modelo PEFT: {peft_accuracy:.2%}")
    print("=" * 80)

    # Mostrar DataFrame
    display(df_report)

else:
    raise ValueError("Las listas de predicciones no tienen la misma longitud.")


COMPARACIÓN DE MODELOS - 20 CASOS DE VALIDACIÓN
Accuracy Modelo Base: 95.00%
Accuracy Modelo PEFT: 95.00%


,TEXTO,ETIQUETA_REAL,BASE_MODEL_PRED,PEFT_QWEN2.5-0.5B_PRED,BASE_CORRECTA,PEFT_CORRECTA
0,Proporcione un ejemplo de un estilo de arte qu...,No,No,No,True,True
1,Describa lo que se está diciendo en la oración.,No,No,No,True,True
2,¿Cuál es la altitud promedio de la Gran Barrer...,No,No,No,True,True
3,Crear un perfil de personaje para una maestra ...,No,No,No,True,True
4,"¿Qué siempre se acumula, pero debe gastarse en...",No,No,No,True,True
5,Extrae dos adjetivos de esta oración:,No,No,No,True,True
6,Sugiere 3 recursos educativos para personas in...,No,No,No,True,True
7,Un científico utilizó su pensamiento creativo ...,No,No,No,True,True
8,Enumera 5 libros favoritos.,No,No,No,True,True
9,Estoy en el mismo barco que tú. Mis padres son...,Sí,Sí,Sí,True,True


In [ ]:
# ============================================================================
# Análisis detallado caso por caso
# ============================================================================

print("\n" + "=" * 80)
print("ANÁLISIS DETALLADO CASO POR CASO")
print("=" * 80)

for i in range(num_samples):
    print(f"\n{'=' * 80}")
    print(f"CASO #{i+1}")
    print(f"{'=' * 80}")
    print(f"TEXTO:\n{texts[i][:200]}...")  # Primeros 200 caracteres
    print(f"{'-' * 80}")
    print(f"ETIQUETA REAL: {human_labels_text[i]} (Label: {human_baseline_labels[i]})")
    print(f"{'-' * 80}")
    print(f"MODELO BASE:")
    print(f"  Predicción: {base_predictions_text[i]}")
    print(f"  Probabilidad clase suicida: {base_model_probabilities[i]:.3f}")
    print(f"  ¿Correcta? {'✓' if df_report.loc[i, 'BASE_CORRECTA'] else '✗'}")
    print(f"{'-' * 80}")
    print(f"MODELO PEFT ({MODEL_NAME.upper()}):")
    print(f"  Predicción: {peft_predictions_text[i]}")
    print(f"  Probabilidad clase suicida: {peft_model_probabilities[i]:.3f}")
    print(f"  ¿Correcta? {'✓' if df_report.loc[i, 'PEFT_CORRECTA'] else '✗'}")


ANÁLISIS DETALLADO CASO POR CASO

CASO #1
TEXTO:
Proporcione un ejemplo de un estilo de arte que utiliza colores brillantes....
--------------------------------------------------------------------------------
ETIQUETA REAL: No (Label: 0)
--------------------------------------------------------------------------------
MODELO BASE:
  Predicción: No
  Probabilidad clase suicida: 0.000
  ¿Correcta? ✓
--------------------------------------------------------------------------------
MODELO PEFT (QWEN2.5-0.5B):
  Predicción: No
  Probabilidad clase suicida: 0.000
  ¿Correcta? ✓

CASO #2
TEXTO:
Describa lo que se está diciendo en la oración....
--------------------------------------------------------------------------------
ETIQUETA REAL: No (Label: 0)
--------------------------------------------------------------------------------
MODELO BASE:
  Predicción: No
  Probabilidad clase suicida: 0.017
  ¿Correcta? ✓
--------------------------------------------------------------------------------
MO

## **Guardar y cargar modelo en HF**

### Guardar Modelo

In [ ]:
# Inicializar API
api = HfApi()

def save_model_locally(model, model_path):
    """Guarda el modelo localmente."""
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    print(f"Modelo guardado localmente en {model_path}.")

def create_or_get_repo(repo_id, token, private=False):
    """Crea un repositorio en Hugging Face Hub si no existe."""
    try:
        api.create_repo(repo_id=repo_id, token=token, private=private, exist_ok=True)
        print(f"Repositorio listo: {repo_id}")
    except Exception as e:
        print(f"Error al crear o acceder al repositorio: {e}")

def upload_lora_adapters(repo_id, lora_model_path, token):
    """Sube los adaptadores LoRA al Hugging Face Hub."""
    api.upload_folder(repo_id=repo_id, folder_path=lora_model_path, token=token)
    print(f"Adaptadores LoRA subidos exitosamente en {repo_id}.")

def save_and_upload_quantized_model(model, model_name, tokenizer, hf_user_name, token, quantization_method="q4_k_m"):
    """Guarda el modelo en formato GGUF cuantizado y lo sube al Hugging Face Hub."""
    model_gguf_path = f"{model_name}-gguf"
    model.save_pretrained_gguf(model_gguf_path, tokenizer, quantization_method=quantization_method)
    model.push_to_hub_gguf(
        repo_id=f"{hf_user_name}/{model_gguf_path}",
        tokenizer=tokenizer,
        quantization_method=quantization_method,
        token=token
    )
    print(f"Modelo GGUF cuantizado subido en {hf_user_name}/{model_gguf_path}.")

In [ ]:
# Configuraciones generales
HF_USERNAME = "Judiht"
HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN")  # Accede al token de Hugging Face desde una variable de entorno
FILE_NAME = "data34K"
REPO_LOCAL_NAME = f"{CLEAN_MODEL_NAME}_{FILE_NAME}_{current_datetime}"
REPO_HF_NAME = f"{HF_USERNAME}/{REPO_LOCAL_NAME}"

# Verificación del token
if HF_TOKEN is None:
    raise ValueError("El token de Hugging Face no está configurado. Asegúrate de definir HUGGING_FACE_HUB_TOKEN en las variables de entorno.")

# Flujo principal
# Guardar el modelo base localmente
save_model_locally(peft_model, REPO_LOCAL_NAME)

# Crear el repositorio en Hugging Face Hub
create_or_get_repo(REPO_HF_NAME, HF_TOKEN)

# Subir los adaptadores LoRA
upload_lora_adapters(REPO_HF_NAME, REPO_LOCAL_NAME, HF_TOKEN)



Modelo guardado localmente en Qwen2_5-0_5B_data34K_20260201_225818.
Repositorio listo: Judiht/Qwen2_5-0_5B_data34K_20260201_225818


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...201_225818/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...adapter_model.safetensors:  51%|#####     |  558kB / 1.10MB            

Adaptadores LoRA subidos exitosamente en Judiht/Qwen2_5-0_5B_data34K_20260201_225818.


### Cargar Modelo

In [ ]:
REPO_HF_NAME = "Judiht/Qwen2_5-0_5B_data34K_20260201_225818"

In [ ]:
# Configuración
#evice = get_device()
config = PeftConfig.from_pretrained(REPO_HF_NAME)

# Cargar el modelo base con el device_map configurado automáticamente
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME, device_map="auto")
optimized_model = PeftModel.from_pretrained(base_model, REPO_HF_NAME)

# Cargar el tokenizador y establecer el pad_token para evitar advertencias
optokenizer = AutoTokenizer.from_pretrained(REPO_HF_NAME, use_fast=True, clean_up_tokenization_spaces=True)
optokenizer.pad_token = optokenizer.eos_token

# Verificación
print("Modelo PEFT optimizado cargado correctamente en el dispositivo.")

adapter_config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/3.21M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

Modelo PEFT optimizado cargado correctamente en el dispositivo.
